In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28*28).astype('float32')/255.0
x_test = x_test.reshape(-1,28*28).astype('float32')/255.0

In [3]:
# class Dense(layers.Layer):
#     def __init__(self, units, input_dim):
#         super().__init__()
#         self.w = self.add_weight(
#                     name='w',
#                     shape=(input_dim, units),
#                     initializer='random_normal',
#                     trainable=True)
#         self.b = self.add_weight(
#                     name='b',
#                     shape=(units,),
#                     initializer='zeros',
#                     trainable=True)
    
#     def call(self, inputs):
#         return tf.matmul(inputs, self.w) + self.b

In [4]:
class MyReLU(layers.Layer):
    def __init__(self):
        super().__init__()
        
    def call(self, x):
        return tf.math.maximum(x,0)

In [5]:
class Dense(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def build(self, input_shape):
        self.w = self.add_weight(
                    name='w',
                    shape=(input_shape[-1], self.units),
                    initializer='random_normal',
                    trainable=True)
        self.b = self.add_weight(
                    name='b',
                    shape=(self.units,),
                    initializer='zeros',
                    trainable=True)       

In [6]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLU()
        
#         self.dense1 = Dense(64,784)  #without build
#         self.dense2 = Dense(10,64)   #without build
        
#         self.dense1 = layers.Dense(64)
#         self.dense2 = layers.Dense(num_classes)
    
    def call(self, input_tensor):
#         x = tf.nn.relu(self.dense1(input_tensor))
        x = self.relu(self.dense1(input_tensor))
        return tf.nn.softmax(self.dense2(x))
    
model = MyModel()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'])

model.fit(x_train,y_train,batch_size=32,epochs=2)
model.evaluate(x_test,y_test,batch_size=32)

Epoch 1/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5922 - accuracy: 0.8462
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 0.1459 - accuracy: 0.9567


[0.14589618146419525, 0.9567000269889832]